In [18]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import struct
from PIL import Image

IMAGE_WIDTH = 1024
IMAGE_HEIGHT = 768
MAX_RAY_DEPTH = 5

def save_bmp(filename, image, width, height):
    with open(filename, 'wb') as f:
        # Заголовок BMP
        bmp_header = bytearray()
        bmp_header.extend(b'BM')  
        bmp_header.extend(struct.pack('<I', 0))
        bmp_header.extend(struct.pack('<H', 0))
        bmp_header.extend(struct.pack('<H', 0))
        bmp_header.extend(struct.pack('<I', 54))
        bmp_header.extend(struct.pack('<I', 40))
        bmp_header.extend(struct.pack('<I', width))
        bmp_header.extend(struct.pack('<I', height))
        bmp_header.extend(struct.pack('<H', 1))
        bmp_header.extend(struct.pack('<H', 24))
        bmp_header.extend(struct.pack('<I', 0))
        bmp_header.extend(struct.pack('<I', 0))
        bmp_header.extend(struct.pack('<I', 0))
        bmp_header.extend(struct.pack('<I', 0))
        bmp_header.extend(struct.pack('<I', 0))
        bmp_header.extend(struct.pack('<I', 0))
        f.write(bmp_header)

        # Записываем пиксели изображения
        for row in range(height - 1, -1, -1):  
            for col in range(width):
                idx = (row * width + col) * 3
                blue, green, red = image[idx], image[idx + 1], image[idx + 2]
                f.write(bytes([blue, green, red]))

# CUDA модуль
mod = SourceModule("""
    #include <math.h>
    #define MAX_RAY_DEPTH 5
    #define PI 3.14159265358979323846

    struct Vector3 {
        float x, y, z;
    
        __device__ Vector3 operator+(const Vector3& v) const {
            return {x + v.x, y + v.y, z + v.z};
        }
    
        __device__ Vector3& operator+=(const Vector3& v) {
            x += v.x;
            y += v.y;
            z += v.z;
            return *this;
        }
    
        __device__ Vector3 operator-(const Vector3& v) const {
            return {x - v.x, y - v.y, z - v.z};
        }
    
        __device__ Vector3 operator*(float scalar) const {
            return {x * scalar, y * scalar, z * scalar};
        }
    
        // Умножение поэлементное для двух векторов
        __device__ Vector3 operator*(const Vector3& v) const {
            return {x * v.x, y * v.y, z * v.z};
        }
    
        __device__ float dot(const Vector3& v) const {
            return x * v.x + y * v.y + z * v.z;
        }
    
        __device__ Vector3 normalize() const {
            float len = sqrtf(dot(*this));
            return (len > 0) ? *this * (1.0f / len) : Vector3{0.0f, 0.0f, 0.0f};
        }
    };

    struct Sphere {
        Vector3 center;
        float radius;
        Vector3 color;
    };

    struct LightSource {
        Vector3 position;
        Vector3 intensity;
    };

    struct Plane {
        Vector3 point;
        Vector3 normal;
        Vector3 color;
    };

    __device__ bool rayIntersect(const Vector3& rayOrigin, const Vector3& rayDir, const Sphere& sphere, float& t) {
        Vector3 oc = rayOrigin - sphere.center;
        float a = rayDir.dot(rayDir);
        float b = 2.0f * oc.dot(rayDir);
        float c = oc.dot(oc) - sphere.radius * sphere.radius;
        float discriminant = b * b - 4 * a * c;

        if (discriminant < 0) return false;
        t = (-b - sqrtf(discriminant)) / (2.0f * a);
        return t >= 0;
    }

    __device__ bool rayIntersectPlane(const Vector3& rayOrigin, const Vector3& rayDir, const Plane& plane, float& t) {
        float denom = plane.normal.dot(rayDir);
        if (fabs(denom) > 1e-6f) {
            Vector3 p0l0 = plane.point - rayOrigin;
            t = p0l0.dot(plane.normal) / denom;
            return (t >= 0);
        }
        return false;
    }

    __device__ Vector3 computeRayColor(const Vector3& rayOrigin, const Vector3& rayDir, Sphere* spheres, int numSpheres, LightSource* lights, int numLights, Plane* planes, int numPlanes, int depth) {
        if (depth > MAX_RAY_DEPTH) return {0.0f, 0.0f, 0.0f};

        float closestT = 1e20f;
        int closestSphere = -1;
        int closestPlane = -1;
        bool hitPlane = false;

        for (int i = 0; i < numSpheres; ++i) {
            float t;
            if (rayIntersect(rayOrigin, rayDir, spheres[i], t) && t < closestT) {
                closestT = t;
                closestSphere = i;
            }
        }

        for (int i = 0; i < numPlanes; ++i) {
            float t;
            if (rayIntersectPlane(rayOrigin, rayDir, planes[i], t) && t < closestT) {
                closestT = t;
                closestPlane = i;
                hitPlane = true;
            }
        }

        Vector3 color = {0.0f, 0.0f, 0.0f};

        if (hitPlane && closestPlane != -1) {
            const Plane& plane = planes[closestPlane];
            Vector3 intersection = rayOrigin + rayDir * closestT;
            Vector3 normal = plane.normal.normalize();

            color = plane.color * 0.1f;

            for (int i = 0; i < numLights; ++i) {
                Vector3 lightDir = (lights[i].position - intersection).normalize();
                float brightness = fmaxf(0.0f, normal.dot(lightDir));
                color += plane.color * (lights[i].intensity * brightness);
            }
        }

        if (closestSphere != -1) {
            const Sphere& sphere = spheres[closestSphere];
            Vector3 intersection = rayOrigin + rayDir * closestT;
            Vector3 normal = (intersection - sphere.center).normalize();

            color = sphere.color * 0.1f;

            for (int i = 0; i < numLights; ++i) {
                Vector3 lightDir = (lights[i].position - intersection).normalize();
                bool inShadow = false;

                for (int j = 0; j < numSpheres; ++j) {
                    float shadowT;
                    if (rayIntersect(intersection + normal * 1e-4f, lightDir, spheres[j], shadowT)) {
                        inShadow = true;
                        break;
                    }
                }
                if (!inShadow) {
                    float brightness = fmaxf(0.0f, normal.dot(lightDir));
                    color += sphere.color * (lights[i].intensity * brightness);
                }
            }
        }

        return color;
    }

__global__ void renderScene(unsigned char* img, int width, int height, Sphere* spheres, int numSpheres, LightSource* lights, int numLights, Plane* planes, int numPlanes) {
    int pixelX = blockIdx.x * blockDim.x + threadIdx.x;
    int pixelY = blockIdx.y * blockDim.y + threadIdx.y;

    if (pixelX >= width || pixelY >= height) return;

    Vector3 rayOrigin = {0.0f, 0.0f, 5.0f};
    Vector3 rayDir = {(2.0f * (pixelX + 0.5f) / width - 1.0f) * width / height, (1.0f - 2.0f * (pixelY + 0.5f) / height), -1.0f};
    rayDir = rayDir.normalize();

    Vector3 color = computeRayColor(rayOrigin, rayDir, spheres, numSpheres, lights, numLights, planes, numPlanes, 0);

    int pixelIdx = (pixelY * width + pixelX) * 3;
    img[pixelIdx] = (unsigned char)(fminf(1.0f, color.x) * 255);
    img[pixelIdx + 1] = (unsigned char)(fminf(1.0f, color.y) * 255);
    img[pixelIdx + 2] = (unsigned char)(fminf(1.0f, color.z) * 255);
}
""")

scene_spheres = np.array([
    (np.array([0.0, -1.0, -3.0], dtype=np.float32), 1.0, np.array([1.0, 0.0, 0.0], dtype=np.float32)),
    (np.array([2.0, 0.0, -4.0], dtype=np.float32), 1.0, np.array([0.0, 1.0, 0.0], dtype=np.float32)),
    (np.array([3.0, -2.0, -3.0], dtype=np.float32), 1.0, np.array([0.0, 0.0, 1.0], dtype=np.float32)),
    (np.array([-2.0, 2.0, -4.0], dtype=np.float32), 1.0, np.array([1.0, 1.0, 0.0], dtype=np.float32)),
    (np.array([-4.0, 3.0, -4.0], dtype=np.float32), 1.0, np.array([0.0, 1.0, 1.0], dtype=np.float32))
], dtype=[('center', np.float32, 3), ('radius', np.float32), ('color', np.float32, 3)])

scene_lights = np.array([
    (np.array([30.0, 5.0, -10.0], dtype=np.float32), np.array([1.0, 1.0, 1.0], dtype=np.float32))
], dtype=[('position', np.float32, 3), ('intensity', np.float32, 3)])

scene_planes = np.array([
    (np.array([0.0, -2.0, -5.0], dtype=np.float32), np.array([0.0, 1.0, 0.0], dtype=np.float32), np.array([0.5, 0.5, 0.5], dtype=np.float32))
], dtype=[('point', np.float32, 3), ('normal', np.float32, 3), ('color', np.float32, 3)])

# Перенос данных на GPU
d_spheres = cuda.mem_alloc(scene_spheres.nbytes)
d_lights = cuda.mem_alloc(scene_lights.nbytes)
d_planes = cuda.mem_alloc(scene_planes.nbytes)
cuda.memcpy_htod(d_spheres, scene_spheres)
cuda.memcpy_htod(d_lights, scene_lights)
cuda.memcpy_htod(d_planes, scene_planes)

# Выделение памяти для изображения
output_image = np.zeros((IMAGE_WIDTH * IMAGE_HEIGHT * 3), dtype=np.uint8)
d_output_image = cuda.mem_alloc(output_image.nbytes)

# Запуск рендеринга
block_size = (16, 16, 1)
grid_size = (IMAGE_WIDTH // block_size[0], IMAGE_HEIGHT // block_size[1])
renderScene = mod.get_function("renderScene")
renderScene(d_output_image, np.int32(IMAGE_WIDTH), np.int32(IMAGE_HEIGHT),
            d_spheres, np.int32(len(scene_spheres)), d_lights, np.int32(len(scene_lights)),
            d_planes, np.int32(len(scene_planes)),
            block=block_size, grid=grid_size)

# Копирование изображения на хост
cuda.memcpy_dtoh(output_image, d_output_image)

# Сохранение изображения
save_bmp('output.bmp', output_image, IMAGE_WIDTH, IMAGE_HEIGHT)